In [1]:
%load_ext sql
import pandas as pd
import numpy as np

In [2]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:85% !important; }</style>"))

![dbSchema](159739A3Schema.jpg)

### High Level Description of [insert Name] Database

#### Players
Stores information about tennis players, including their unique PlayerID, Name, Ranking, Points, and details about whether they have wealthy parents and if their parents are mentioned on Wikipedia.

#### Matches
This table contains details about individual tennis matches, including MatchID, LocationID, TournamentID, CourtID, SurfaceID, RoundID, BestOf (the number of sets required to win the match), WLRank (the opponent's ranking), WLsets (the number of sets won and lost), Comment, AvgWL (average winning likelihood), and Win (indicating if the player won the match).

#### Locations
Represents the locations where tennis matches take place. It includes LocationID and Name.

#### Tournaments
Stores information about tennis tournaments; TournamentID and Name.

#### Tiers
Hold Tier information (the level or category of the tournament, such as International, Grand Slam, etc.)

#### Courts
Describes the types of courts used in tennis matches. It includes CourtID and Type (e.g., Outdoor, Indoor).

#### Surfaces
Contains information about the playing surfaces used in tennis matches, such as SurfaceID and Type (e.g., Hard, Clay, Grass).

#### Rounds
Represents the different rounds or stages within a tennis tournament. It includes RoundID and Name (e.g., 1st Round, Quarterfinals, Finals).

#### PlayerMatches
This table establishes the one-to-many relationships between players and matches. It includes PlayerMatchID (a unique identifier for the relationship), PlayerID (referencing the Players table), and MatchID (referencing the Matches table.

## Connect to DB and Create Tables

In [3]:
%sql sqlite:///./158739Assignment3

In [4]:
%%sql sqlite:///./158739Assignment3

DROP TABLE IF EXISTS Courts;
CREATE TABLE "Courts" (
"CourtID" INTEGER NOT NULL UNIQUE,
"Type" TEXT,
PRIMARY KEY("CourtID" AUTOINCREMENT)
)

Done.
Done.


[]

In [5]:
%%sql sqlite:///./158739Assignment3

DROP TABLE IF EXISTS Locations;

CREATE TABLE "Locations" (
"LocationID" INTEGER NOT NULL UNIQUE,
"Name" TEXT,
PRIMARY KEY("LocationID" AUTOINCREMENT)
)

Done.
Done.


[]

In [6]:

%%sql sqlite:///./158739Assignment3

DROP TABLE IF EXISTS Matches;
    CREATE TABLE "Matches" (
"MatchID"	INTEGER NOT NULL UNIQUE,
"LocationID"	INTEGER,
"TournamentID"	INTEGER,
"CourtID"	INTEGER,
"SurfaceID"	INTEGER,
"RoundID"	INTEGER,
"BestOf"	INTEGER,
"WLRank"	INTEGER,
"WLsets"	INTEGER,
"Comment"	TEXT,
"AvgWL"	REAL,
"Win"	INTEGER,
FOREIGN KEY("PlayerID") REFERENCES "Players"("PlayerID"),
PRIMARY KEY("MatchID" AUTOINCREMENT)
)

Done.
Done.


[]

## [insert name] Database Schema

In [7]:
%%sql sqlite:///./158739Assignment3

DROP TABLE IF EXISTS PlayerMatches;
CREATE TABLE "PlayerMatches" (
"PlayerMatchID"	INTEGER NOT NULL UNIQUE,
"PlayerID"	INTEGER,
"MatchID"	INTEGER,
FOREIGN KEY("PlayerID") REFERENCES "Players"("PlayerID"),
FOREIGN KEY("MatchID") REFERENCES "Matches"("MatchID"),
PRIMARY KEY("PlayerMatchID" AUTOINCREMENT)
)

Done.
Done.


[]

In [8]:
%%sql sqlite:///./158739Assignment3

DROP TABLE IF EXISTS Players;
CREATE TABLE "Players" (
"PlayerID"	INTEGER NOT NULL UNIQUE,
"Name"	TEXT,
"Ranking"	INTEGER,
"WealthyParents"	INTEGER,
"ParentOnWiki"	INTEGER,
PRIMARY KEY("PlayerID" AUTOINCREMENT)
)

Done.
Done.


[]

In [9]:

%%sql sqlite:///./158739Assignment3

DROP TABLE IF EXISTS Rounds;
CREATE TABLE "Rounds" (
"RoundID"	INTEGER NOT NULL UNIQUE,
"Name"	TEXT,
PRIMARY KEY("RoundID" AUTOINCREMENT)
)

Done.
Done.


[]

In [10]:
%%sql sqlite:///./158739Assignment3

DROP TABLE IF EXISTS Surfaces;
CREATE TABLE "Surfaces" (
"SurfaceID"	INTEGER NOT NULL UNIQUE,
"Type"	TEXT,
PRIMARY KEY("SurfaceID" AUTOINCREMENT)
)

Done.
Done.


[]

In [11]:

%%sql sqlite:///./158739Assignment3

DROP TABLE IF EXISTS Tournaments;

CREATE TABLE "Tournaments" (
"TournamentID"	INTEGER NOT NULL UNIQUE,
"Name"	TEXT,
PRIMARY KEY("TournamentID" AUTOINCREMENT)
)



Done.
Done.


[]

In [12]:
%%sql sqlite:///./158739Assignment3

DROP TABLE IF EXISTS Tier;

CREATE TABLE "Tier" (
    "TierID" INTEGER NOT NULL UNIQUE,
    "Name" TEXT,
    PRIMARY KEY("TierID" AUTOINCREMENT)
);

Done.
Done.


[]

## Import data into tables

In [13]:
# import CSV into a dataframe
# or get top_players from csv, to reduce API calls
player_odds_df = pd.read_csv("all_matches_player_odds.csv", index_col=0)
pd.set_option('display.max_columns', None)
player_odds_df.head()


,WTA,Location,Tournament,Date,Tier,Court,Surface,Round,Best of,WinLoseName,WLRank,WPts,WLsets,Comment,AvgWL,ATP,Series,win,LPts,Name,Rank,wealthy_parents,parent_on_wiki
0,1.0,Auckland,ASB Classic,31/12/2018,International,Outdoor,Hard,1st Round,3.0,Puig M.,53.0,1010.0,2.0,Completed,1.37,0.0,0,True,0.0,NaN,0.0,Flase,False
1,1.0,Auckland,ASB Classic,31/12/2018,International,Outdoor,Hard,1st Round,3.0,Davis L.,173.0,340.0,2.0,Completed,1.50,0.0,0,True,0.0,Lauren Davis,58.0,False,True
2,1.0,Auckland,ASB Classic,31/12/2018,International,Outdoor,Hard,1st Round,3.0,Kuzmova V.,50.0,1052.0,2.0,Completed,1.62,0.0,0,True,0.0,NaN,0.0,Flase,False
3,1.0,Auckland,ASB Classic,31/12/2018,International,Outdoor,Hard,1st Round,3.0,Bouchard E.,87.0,692.0,2.0,Completed,1.53,0.0,0,True,0.0,NaN,0.0,Flase,False
4,1.0,Auckland,ASB Classic,31/12/2018,International,Outdoor,Hard,1st Round,3.0,Sorribes Tormo S.,86.0,700.0,2.0,Completed,2.35,0.0,0,True,0.0,NaN,0.0,Flase,False


In [15]:
# populate players table

unique_Name = pd.DataFrame(player_odds_df['WinLoseName'].drop_duplicates())
unique_Rank = pd.DataFrame(player_odds_df['Rank'].drop_duplicates())
unique_WealthyParents = pd.DataFrame(player_odds_df['wealthy_parents'].drop_duplicates())
unique_ParentOnWiki = pd.DataFrame(player_odds_df['parent_on_wiki'].drop_duplicates())

names = unique_Name['WinLoseName'].tolist()
ranks = unique_Rank['Rank'].tolist()
wealthyparents = unique_WealthyParents['wealthy_parents'].tolist()
parentonwikis = unique_ParentOnWiki['parent_on_wiki'].tolist()

%sql DELETE FROM Tournaments

for name in names:
    %sql INSERT INTO Players (Name) VALUES (:name)
    
for rank in ranks:
    %sql INSERT INTO Players (Ranking) VALUES (:rank)
    
for parents in wealthyparents:
    %sql INSERT INTO Players (WealthyParents) VALUES (:parents)
    
for parents in parentonwikis:
    %sql INSERT INTO Players (ParentOnWiki) VALUES (:parents)

 * sqlite:///./158739Assignment3
0 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.


1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3


1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3


1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3


1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3


...

1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3


1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3


1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3


1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3


1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3
1 rows affected.
 * sqlite:///./158739Assignment3


In [19]:
player_odds_df.columns

Index(['WTA', 'Location', 'Tournament', 'Date', 'Tier', 'Court', 'Surface',
       'Round', 'Best of', 'WinLoseName', 'WLRank', 'WPts', 'WLsets',
       'Comment', 'AvgWL', 'ATP', 'Series', 'win', 'LPts', 'Name', 'Rank',
       'wealthy_parents', 'parent_on_wiki'],
      dtype='object')

In [18]:
# populate matches table

unique_bestof = pd.DataFrame(player_odds_df['Best of'].drop_duplicates())
unique_WLrank = pd.DataFrame(player_odds_df['WLRank'].drop_duplicates())
unique_wlsets = pd.DataFrame(player_odds_df['WLsets'].drop_duplicates())
unique_comment = pd.DataFrame(player_odds_df['Comment'].drop_duplicates())
unique_avgwl = pd.DataFrame(player_odds_df['AvgWL'].drop_duplicates())
unique_win = pd.DataFrame(player_odds_df['win'].drop_duplicates())
                                   
bestof = unique_bestof['Best of'].tolist()
wlrank = unique_WLrank['WLRank'].tolist()
wlsets = unique_wlsets['WLsets'].tolist()
comment = unique_comment['wealthy_parents'].tolist()
avgwl = unique_avgwl['AvgWL'].tolist()
wins = unique_win['win'].tolist()

%sql DELETE FROM Matches

for bests in bestof:
    %sql INSERT INTO Players (BestOf) VALUES (:bests)
    
for rank in wlrank:
    %sql INSERT INTO Players (WLRank) VALUES (:rank)
    
for wlset in wlsets:
    %sql INSERT INTO Players (WLsets) VALUES (:wlset)
    
for comment in comments:
    %sql INSERT INTO Players (Comment) VALUES (:comment)
    
for avg in avgwl:
    %sql INSERT INTO Players (AvgWL) VALUES (:avg)
                            
for win in wins:
    %sql INSERT INTO Players (Win) VALUES (:win)

KeyError: 'Best of'

In [ ]:
# populate tournaments table

unique_tournaments = pd.DataFrame(player_odds_df['Tournament'].drop_duplicates())

tournaments = unique_tournaments['Tournament'].tolist()

%sql DELETE FROM Tournaments

for tournament in tournaments:
    %sql INSERT INTO Tournaments (Name) VALUES (:tournament)

In [ ]:
# populate tier table

unique_tiers = pd.DataFrame(player_odds_df['Tier'].drop_duplicates())

tiers = unique_tiers['Tier'].tolist()

%sql DELETE FROM Tier

for tier in tiers:
    %sql INSERT INTO Tournaments (Name) VALUES (:tier)

In [ ]:
# populate courts table
1
unique_values_df = pd.DataFrame(player_odds_df['Court'].drop_duplicates())
unique_ids = pd.RangeIndex(start=1, stop=len(unique_values_df) + 1)
unique_values_df.insert(0, 'ID', unique_ids)

data = unique_values_df.values.tolist()

%sql DELETE FROM Courts

for row in data:
    trow = tuple(row)
    sql = "INSERT INTO Courts VALUES {}".format(trow)  
    %sql $sql

In [ ]:
# populate Locations table
unique_values_df = pd.DataFrame(player_odds_df['Location'].drop_duplicates())
unique_ids = pd.RangeIndex(start=1, stop=len(unique_values_df) + 1)
unique_values_df.insert(0, 'ID', unique_ids)

data = unique_values_df.values.tolist()

%sql DELETE FROM Locations

for row in data:
    trow = tuple(row)
    sql = "INSERT INTO Locations VALUES {}".format(trow)  
    %sql $sql
    

In [ ]:
# populate Surfaces table
unique_values_df = pd.DataFrame(player_odds_df['Surface'].drop_duplicates())
unique_ids = pd.RangeIndex(start=1, stop=len(unique_values_df) + 1)
unique_values_df.insert(0, 'ID', unique_ids)

data = unique_values_df.values.tolist()

%sql DELETE FROM Locations

for row in data:
    trow = tuple(row)
    sql = "INSERT INTO Surfaces VALUES {}".format(trow)  
    %sql $sql

In [ ]:
# populate Rounds table
unique_values_df = pd.DataFrame(player_odds_df['Round'].drop_duplicates())
unique_ids = pd.RangeIndex(start=1, stop=len(unique_values_df) + 1)
unique_values_df.insert(0, 'ID', unique_ids)

data = unique_values_df.values.tolist()

%sql DELETE FROM Locations

for row in data:
    trow = tuple(row)
    sql = "INSERT INTO Rounds VALUES {}".format(trow)  
    %sql $sql